In [1]:
import torch
from transformers import AutoTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup,get_polynomial_decay_schedule_with_warmup, Adafactor
import torch.nn as nn
import logging
from lion_pytorch import Lion
from torch.utils.data import DataLoader
import GLUEforQQP
from torch.optim.lr_scheduler import LambdaLR
from functools import partial
import numpy as np
from sklearn.metrics import matthews_corrcoef
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:

dataset_name = 'qqp'
current_path = Path.cwd().parents[0]/dataset_name
current_path.mkdir(exist_ok=True)
# lr_list = [3e-5,3e-4,3e-3]
# scheduler_list = ['no', 'linear']
# optimizer_list = ['Lion', 'AdamW','AdaFactor']
# batch_size_list = [32,64,128]
# steps = 50*1000
lr_list = [3e-5,3e-6,3e-7]
scheduler_list = ['no','linear']
optimizer_list = ['Lion']
batch_size_list = [16,32,64]
steps = 20*1000
report_step = 250

In [3]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
train_dataset,_,test_dataset = GLUEforQQP.get_torch_dataset(tokenizer, "qqp", padding="max_length", truncation=True, max_length = 128)

def constant_scheduler(
    optimizer, num_warmup_steps, num_training_steps, lr_end=1e-7, power=1.0, last_epoch=-1
):
    def lambda_func(step:int):
        return 1.

    return LambdaLR(optimizer, lambda_func, last_epoch)

def prepare(sche, opt):
    if sche == 'no':
        sches = partial(constant_scheduler)
    if sche == 'linear':
        sches = partial(get_linear_schedule_with_warmup)

    if opt == 'Lion':
        opts = partial(Lion, betas = (0.95,0.98), weight_decay = 0.01)
    if opt == 'AdaFactor':
        opts = partial(Adafactor, weight_decay = 0.001, relative_step = False, scale_parameter=False)
    if opt == 'AdamW':
        opts = partial(torch.optim.AdamW, betas = (0.9,0.99), weight_decay = 0.001)


    return sches, opts

Found cached dataset glue (C:/Users/Xiang/.cache/huggingface/datasets/mariosasko___glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\Xiang\.cache\huggingface\datasets\mariosasko___glue\cola\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-2bc0360bf4726f4f.arrow
Loading cached processed dataset at C:\Users\Xiang\.cache\huggingface\datasets\mariosasko___glue\cola\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-77227c16e4defc4f.arrow
Loading cached processed dataset at C:\Users\Xiang\.cache\huggingface\datasets\mariosasko___glue\cola\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-c61c89821cd722e6.arrow


In [4]:


def get_log(file_name):
    logger = logging.getLogger('train')  # 设定logger的名字
    logger.setLevel(logging.INFO)  # 设定logger得等级

    ch = logging.StreamHandler()  # 输出流的hander，用与设定logger的各种信息
    ch.setLevel(logging.INFO)  # 设定输出hander的level

    fh = logging.FileHandler(file_name, mode='a')  # 文件流的hander，输出得文件名称，以及mode设置为覆盖模式
    fh.setLevel(logging.INFO)  # 设定文件hander得lever

    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    ch.setFormatter(formatter)  # 两个hander设置个是，输出得信息包括，时间，信息得等级，以及message
    fh.setFormatter(formatter)
    logger.addHandler(fh)  # 将两个hander添加到我们声明的logger中去
    logger.addHandler(ch)
    return logger

In [5]:
logger = get_log('logQQP.txt')

In [6]:
# test_loader = DataLoader(test_dataset, shuffle = False, batch_size = 32)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [7]:
def evaluate(model, dataset):
    model.eval()
    eval_loader = DataLoader(dataset, shuffle = False, batch_size = 32)
    logits = []
    labelss = []
    with torch.no_grad():
        for X in eval_loader:
            batch = {k: v.to(device) for k, v in X.items()}
            logits.append(model(**batch).logits)
            labelss.append(batch['labels'])
        total_test = torch.concat(logits, dim = 0)
        _,predicted = torch.max(total_test,dim = 1)
        real_label =torch.concat(labelss,dim=0).cpu().numpy()
        predicted = predicted.cpu().numpy()
        metric = matthews_corrcoef(real_label, predicted)
        acc = np.mean(predicted==real_label)

    return metric, acc

In [8]:
# Training

# loss_mat = np.zeros((len(batch_size_list),len(scheduler_list), len(optimizer_list), len(lr_list),steps))

 # evaluate test metric each step
# metric_mat = np.zeros((len(batch_size_list),len(scheduler_list), len(optimizer_list), len(lr_list),steps//report_step,2))
for i,this_batch_size in enumerate(batch_size_list):
    for j,this_scheduler in enumerate(scheduler_list):
        for k,this_optimizer in enumerate(optimizer_list):
            for m, this_lr in enumerate(lr_list):
                loss_list = []
                metric_list = []
                acc_list = []
                train_metric_list = []
                train_acc_list = []
                model = BertForSequenceClassification.from_pretrained("bert-base-uncased").to(device)
                train_loader = DataLoader(train_dataset, batch_size=this_batch_size)
                sche, opt = prepare(this_scheduler, this_optimizer)
                optimizer = opt(model.parameters(), lr = this_lr if this_scheduler == 'no' else this_lr*2)
                scheduler = sche(optimizer, num_warmup_steps=int(steps/10),num_training_steps=steps)
                step = 0

                metric,acc = evaluate(model,test_dataset)
                # print(i,j,k,m,step//report_step)
                metric_list.append(metric)
                acc_list.append(acc)
                tmetric, tacc = evaluate(model, train_dataset)
                train_metric_list.append(tmetric)
                train_acc_list.append(tacc)
                # metric_mat[i,j,k,m,step//report_step - 1,0] = metric
                # metric_mat[i,j,k,m,step//report_step - 1,1] = acc
                print(f"step:{step}, matthews_corr:{metric:.6f}, Acc:{acc*100:4f}%, Train: matthews_corr:{tmetric:.6f}, Acc:{tacc*100:4f}%,")

                # print(f'Start training for: sche:{this_scheduler},opt:{this_optimizer},batchsize:{this_batch_size}, lr:{this_lr}')
                logger.info(f'Start training for: sche:{this_scheduler},opt:{this_optimizer},batchsize:{this_batch_size}, lr:{this_lr}')
                while True:

                    for X in train_loader:
                        model.train()
                        optimizer.zero_grad()
                        batch = {k: v.to(device) for k, v in X.items()}
                        loss = model(**batch).loss
                        print(f"step: {step+1}, loss:{loss.item():.8f}")

                        # loss_mat[i,j,k,m,step] = loss.item()
                        loss_list.append(loss.item())
                        loss.backward()
                        optimizer.step()
                        scheduler.step()
                        step += 1

                    # valid
                        if step % report_step == 0:

                            metric,acc = evaluate(model,test_dataset)
                            # print(i,j,k,m,step//report_step)
                            metric_list.append(metric)
                            acc_list.append(acc)
                            tmetric, tacc = evaluate(model, train_dataset)
                            train_metric_list.append(tmetric)
                            train_acc_list.append(tacc)
                            # metric_mat[i,j,k,m,step//report_step - 1,0] = metric
                            # metric_mat[i,j,k,m,step//report_step - 1,1] = acc
                            # print(f"step:{step}, matthews_corr:{metric:.6f}, Acc:{acc*100:4f}%")
                            logger.info(f"step:{step}, matthews_corr:{metric:.6f}, Acc:{acc*100:4f}%, Train: matthews_corr:{tmetric:.6f}, Acc:{tacc*100:4f}%,")

                        if step == steps:
                            break
                    if step == steps:
                        break
                file_name = dataset_name+",batchsize"+str(this_batch_size)+",scheduler"+this_scheduler+",optimizer"+str(this_optimizer)+",LR"+str(this_lr)
                np.save(current_path/(file_name+'loss.npy'),np.array(loss_list))
                np.save(current_path/(file_name+'metric.npy'),np.array(metric_list))
                np.save(current_path/(file_name+'acc.npy'),np.array(acc_list))
                np.save(current_path/(file_name+'trainmetric.npy'),np.array(train_metric_list))
                np.save(current_path/(file_name+'trainacc.npy'),np.array(train_acc_list))

                del model
                del optimizer
                del scheduler
                del train_loader
                torch.cuda.empty_cache()








Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

step:0, matthews_corr:0.000000, Acc:69.127517%, Train: matthews_corr:0.000000, Acc:70.436206%,
step: 1, loss:0.61034375
step: 2, loss:0.66064429
step: 3, loss:0.59298635
step: 4, loss:0.64480376
step: 5, loss:0.73384029
step: 6, loss:0.67509502
step: 7, loss:0.69257718
step: 8, loss:0.61417204
step: 9, loss:0.66316622
step: 10, loss:0.68851578
step: 11, loss:0.60336715
step: 12, loss:0.59988374
step: 13, loss:0.67533374
step: 14, loss:0.66408962
step: 15, loss:0.62408823
step: 16, loss:0.59205210
step: 17, loss:0.63761103
step: 18, loss:0.41686618
step: 19, loss:0.71400911
step: 20, loss:0.53120041
step: 21, loss:0.52699929
step: 22, loss:0.60595208
step: 23, loss:0.83450842
step: 24, loss:0.80086613
step: 25, loss:0.67975205
step: 26, loss:0.72908628
step: 27, loss:0.74066955
step: 28, loss:0.76617563
step: 29, loss:0.73170388
step: 30, loss:0.67078042
step: 31, loss:0.70432043
step: 32, loss:0.64583963
step: 33, loss:0.61035454
step: 34, loss:0.64525199
step: 35, loss:0.63614368
step

2023-03-14 22:01:06,321 - INFO - step:50, matthews_corr:0.101579, Acc:69.606903%, Train: matthews_corr:0.110180, Acc:70.974155%,


step: 51, loss:0.57804012
step: 52, loss:0.85394692
step: 53, loss:0.72642255
step: 54, loss:0.68567646
step: 55, loss:0.62700123
step: 56, loss:0.68476170
step: 57, loss:0.65552586
step: 58, loss:0.66478032
step: 59, loss:0.71052688
step: 60, loss:0.67145962
step: 61, loss:0.67112565
step: 62, loss:0.66009057
step: 63, loss:0.50286579
step: 64, loss:0.73750454
step: 65, loss:0.59366965
step: 66, loss:0.60164285
step: 67, loss:0.51865393
step: 68, loss:0.58071423
step: 69, loss:0.56763107
step: 70, loss:0.40209579
step: 71, loss:0.39870575
step: 72, loss:0.93943655
step: 73, loss:0.70608222
step: 74, loss:0.61325645
step: 75, loss:0.45463094
step: 76, loss:0.56605810
step: 77, loss:0.49052414
step: 78, loss:0.52445620
step: 79, loss:0.67862177
step: 80, loss:0.90745294
step: 81, loss:0.73731112
step: 82, loss:0.73163581
step: 83, loss:0.56999749
step: 84, loss:0.68587291
step: 85, loss:0.59865040
step: 86, loss:0.75478929
step: 87, loss:0.64339650
step: 88, loss:0.67281717
step: 89, lo

2023-03-14 22:01:32,682 - INFO - step:100, matthews_corr:0.077385, Acc:69.319271%, Train: matthews_corr:0.161616, Acc:71.722606%,


...

2023-03-14 22:34:59,424 - INFO - step:3900, matthews_corr:0.000000, Acc:69.127517%, Train: matthews_corr:0.000000, Acc:70.436206%,


step: 3901, loss:0.60204685
step: 3902, loss:0.52112347
step: 3903, loss:0.65374964
step: 3904, loss:0.62073421
step: 3905, loss:0.62827826
step: 3906, loss:0.60160834
step: 3907, loss:0.59893489
step: 3908, loss:0.64026779
step: 3909, loss:0.56733865
step: 3910, loss:0.47420472
step: 3911, loss:0.49171406
step: 3912, loss:0.64696908
step: 3913, loss:0.62879610
step: 3914, loss:0.49318722
step: 3915, loss:0.44213614
step: 3916, loss:0.56773865
step: 3917, loss:0.56535256
step: 3918, loss:0.69259936
step: 3919, loss:0.55710655
step: 3920, loss:0.56739885
step: 3921, loss:0.67677963
step: 3922, loss:0.72227633
step: 3923, loss:0.69838113
step: 3924, loss:0.50009018
step: 3925, loss:0.67380667
step: 3926, loss:0.66967160
step: 3927, loss:0.57098705
step: 3928, loss:0.56883287
step: 3929, loss:0.54166412
step: 3930, loss:0.41882643
step: 3931, loss:0.43997163
step: 3932, loss:0.39250880
step: 3933, loss:0.46321982
step: 3934, loss:0.49159023
step: 3935, loss:0.51934952
step: 3936, loss:0.5

2023-03-14 22:35:25,374 - INFO - step:3950, matthews_corr:0.000000, Acc:69.127517%, Train: matthews_corr:0.000000, Acc:70.436206%,


step: 3951, loss:0.57151699
step: 3952, loss:0.46727756
step: 3953, loss:0.58321941
step: 3954, loss:0.73407221
step: 3955, loss:0.48778996
step: 3956, loss:0.59803212
step: 3957, loss:0.88457680
step: 3958, loss:0.56756687
step: 3959, loss:0.54327530
step: 3960, loss:0.43179220
step: 3961, loss:0.35694271
step: 3962, loss:0.43927544
step: 3963, loss:0.38336480
step: 3964, loss:0.94475800
step: 3965, loss:0.53499657
step: 3966, loss:0.56904393
step: 3967, loss:0.53725356
step: 3968, loss:0.35494062
step: 3969, loss:0.48886490
step: 3970, loss:0.37649792
step: 3971, loss:0.48659286
step: 3972, loss:0.68220246
step: 3973, loss:0.59070045
step: 3974, loss:0.55038768
step: 3975, loss:0.81446570
step: 3976, loss:0.46575069
step: 3977, loss:0.70966923
step: 3978, loss:0.66456360
step: 3979, loss:0.43579301
step: 3980, loss:0.56404030
step: 3981, loss:0.48535272
step: 3982, loss:0.63189846
step: 3983, loss:0.51305985
step: 3984, loss:0.54620969
step: 3985, loss:0.70714110
step: 3986, loss:0.7

2023-03-14 22:35:51,286 - INFO - step:4000, matthews_corr:0.000000, Acc:69.127517%, Train: matthews_corr:0.000000, Acc:70.436206%,
